# How to set a prior given T = 2 non-zero effects?

Assume two non-zero effects $T = 2$. We set PVE $ = 0.05$ so that SuSiE has certain power around $0.38$. 
We also set PVE $ = 0.7$ so that SuSiE has power around $0.64$. We investigate SuSiE performance under different priors.

## Results

** - Summary: prior has a modest influence on SuSiE's performance. **

When $pve = 0.05$, 

* a larger prior has a slighler lower FDR and a slightly higher top_hit_rate.

When $pve = 0.7$,

* a larger prior has a slightly larger power and lower FDR. And cs_size will increase given a larger prior. 


In [7]:
dscout.summary[dscout.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
20,2,0.05,0.01,0.34,0.0000,2.117647,1,0.9412,0.9291165
86,2,0.05,0.02,0.36,0.0526,1.842105,1,0.8684,0.9385918
152,2,0.05,0.03,0.37,0.0513,1.666667,1,0.8718,0.9429208
218,2,0.05,0.05,0.38,0.0500,1.550000,1,0.8750,0.9522987
284,2,0.05,0.10,0.38,0.0500,1.450000,1,0.9000,0.9609036
350,2,0.05,0.20,0.38,0.0500,1.425000,1,0.9000,0.9627169
416,2,0.05,0.40,0.38,0.0500,1.400000,1,0.9000,0.9627169
482,2,0.05,0.50,0.38,0.0500,1.375000,1,0.9000,0.9686246
548,2,0.05,0.70,0.38,0.0500,1.375000,1,0.9000,0.9686246
614,2,0.05,0.90,0.38,0.0500,1.375000,1,0.9000,0.9686246


In [8]:
dscout.summary[dscout.summary$pve==0.7,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
50,2,0.7,0.01,0.63,0.0156,1.06,1.28,0.9688,0.9979029
116,2,0.7,0.02,0.63,0.0156,1.09,1.28,0.9688,0.9969122
182,2,0.7,0.03,0.63,0.0156,1.11,1.28,0.9688,0.9969122
248,2,0.7,0.05,0.63,0.0156,1.14,1.28,0.9688,0.9969122
314,2,0.7,0.10,0.63,0.0000,1.02,1.26,0.9841,0.9994104
380,2,0.7,0.20,0.64,0.0000,1.56,1.28,0.9688,0.9958743
446,2,0.7,0.40,0.64,0.0000,1.48,1.28,0.9688,0.9959935
512,2,0.7,0.50,0.64,0.0000,1.47,1.28,0.9688,0.9959935
578,2,0.7,0.70,0.64,0.0000,1.46,1.28,0.9688,0.9959935
644,2,0.7,0.90,0.64,0.0000,1.46,1.28,0.9688,0.9959935


## Code details

In [2]:
dscout_Q2 = readRDS('dscout_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [3]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [4]:
hitsum.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, mean)
power.summary = hitmean.summary
power.summary$power = power.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
cs_num.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, meannonzero)
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, dscout_df, meannonzero)

In [5]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, 
                            setsize.summary$cs_size, cs_num.summary$cs_num, 
                            tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [6]:
dscout.summary = dscout.summary[dscout.summary$effect_num==2, ]